# Téléchargement des fichiers ZIP Binance Klines BTCUSDT 5m

Ce notebook télécharge tous les fichiers ZIP de la page [Binance Vision - BTCUSDT 5m](https://data.binance.vision/?prefix=data/spot/monthly/klines/BTCUSDT/5m/).

- **Paramétrable** : marché, granularité, type de données, symbole, etc.
- **Téléchargement parallèle** (max 5/minute)
- **Vérification d'intégrité**
- **Destination** : `/home/giujorge/Downloads/`

---

In [1]:
import os
import time
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from datetime import datetime

def generate_zip_links(symbol, interval, start_year, start_month, end_year, end_month):
    """
    Génère la liste des URLs .zip à télécharger entre deux dates (incluses).
    """
    links = []
    y, m = start_year, start_month
    while (y < end_year) or (y == end_year and m <= end_month):
        fname = f"{symbol}-{interval}-{y:04d}-{m:02d}.zip"
        url = f"https://data.binance.vision/data/spot/monthly/klines/{symbol}/{interval}/{fname}"
        links.append(url)
        m += 1
        if m > 12:
            m = 1
            y += 1
    return links


In [46]:
# Paramètres variables
DATA_TYPE = "data"
MARKET = "spot"
DATA_FREQUENCY = "monthly"
DATA_CATEGORY = "klines"
SYMBOL = "BTCUSDT"
INTERVAL = "4h"
DEST_DIR = f"/media/giujorge/Stockage/DATA/raw/binance/{MARKET}/{DATA_FREQUENCY}/{DATA_CATEGORY}/{SYMBOL}/{INTERVAL}"
MAX_PARALLEL = 5
RATE_LIMIT_SECONDS = 60  # 5 téléchargements par minute
START_YEAR, START_MONTH = 2017, 8
END_YEAR, END_MONTH = 2025, 8

BASE_URL = f"https://data.binance.vision/{DATA_TYPE}/{MARKET}/{DATA_FREQUENCY}/{DATA_CATEGORY}/{SYMBOL}/{INTERVAL}/"

print(f"Cible: {BASE_URL.format({SYMBOL}, {INTERVAL})}")

# Création du dossier de destination
os.makedirs(DEST_DIR, exist_ok=True)


Cible: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/4h/


In [47]:
def download_file(url, dest_dir):
    """Télécharge un fichier ZIP et vérifie son intégrité."""
    local_filename = os.path.join(dest_dir, url.split('/')[-1])
    if os.path.exists(local_filename):
        return local_filename, 'déjà téléchargé'
    try:
        with requests.get(url, stream=True, timeout=60) as r:
            if r.status_code == 404:
                return local_filename, '404 not found'
            r.raise_for_status()
            with open(local_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
        # Vérification basique de l'intégrité (taille > 0)
        if os.path.getsize(local_filename) == 0:
            os.remove(local_filename)
            return local_filename, 'fichier vide'
        return local_filename, 'ok'
    except Exception as e:
        if os.path.exists(local_filename):
            os.remove(local_filename)
        return local_filename, f'erreur: {e}'


In [48]:
zip_links = generate_zip_links(SYMBOL, INTERVAL, START_YEAR, START_MONTH, END_YEAR, END_MONTH)
print(f"{len(zip_links)} fichiers .zip à télécharger.")
if zip_links:
    print("Exemple de lien:", zip_links[0])


97 fichiers .zip à télécharger.
Exemple de lien: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/4h/BTCUSDT-4h-2017-08.zip


In [49]:
# Téléchargement avec ThreadPoolExecutor et limitation de débit
results = []
for i in range(0, len(zip_links), MAX_PARALLEL):
    batch = zip_links[i:i+MAX_PARALLEL]
    with ThreadPoolExecutor(max_workers=MAX_PARALLEL) as executor:
        future_to_url = {executor.submit(download_file, url, DEST_DIR): url for url in batch}
        for future in tqdm(as_completed(future_to_url), total=len(batch), desc=f"Batch {i//MAX_PARALLEL+1}"):
            url = future_to_url[future]
            try:
                filename, status = future.result()
                results.append((filename, status))
            except Exception as exc:
                results.append((url, f'erreur: {exc}'))
    if i + MAX_PARALLEL < len(zip_links):
        print(f"Pause {RATE_LIMIT_SECONDS}s pour respecter la limite de 5 téléchargements/minute...")
        time.sleep(RATE_LIMIT_SECONDS)

Batch 1:   0%|          | 0/5 [00:00<?, ?it/s]

Batch 1: 100%|██████████| 5/5 [00:00<00:00,  5.08it/s]


Pause 60s pour respecter la limite de 5 téléchargements/minute...


Batch 2: 100%|██████████| 5/5 [00:01<00:00,  2.86it/s]


Pause 60s pour respecter la limite de 5 téléchargements/minute...


...

Batch 18: 100%|██████████| 5/5 [00:00<00:00,  5.04it/s]


Pause 60s pour respecter la limite de 5 téléchargements/minute...


Batch 19: 100%|██████████| 5/5 [00:01<00:00,  4.15it/s]


Pause 60s pour respecter la limite de 5 téléchargements/minute...


Batch 20: 100%|██████████| 2/2 [00:01<00:00,  1.81it/s]


In [50]:
# Résumé
ok = sum(1 for _, s in results if s == 'ok')
erreurs = [(f, s) for f, s in results if s != 'ok' and s != 'déjà téléchargé']
print(f"\nTéléchargements réussis: {ok}/{len(zip_links)}")
if erreurs:
    print("Erreurs:")
    for f, s in erreurs:
        print(f"- {f}: {s}")
else:
    print("Aucune erreur détectée.")


Téléchargements réussis: 97/97
Aucune erreur détectée.
